Таска 4

In [1]:
! pip install --upgrade datasets fsspec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 6.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pla

In [2]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer, InputExample, losses, util
from torch.utils.data import DataLoader
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import random
import os

os.environ["WANDB_DISABLED"] = "true"
print("Weights & Biases отключен.")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Используется устройство: {device}")
print("Загрузка датасета sentence-transformers/natural-questions...")
try:
    dataset = load_dataset("sentence-transformers/natural-questions", split='train')
    print("Датасет загружен.")
except ValueError as e:
    print(f"Ошибка при загрузке датасета: {e}")
    print("Возможно, требуется обновить библиотеки datasets и fsspec: pip install --upgrade datasets fsspec")
    raise

print(f"Загружен сплит с {len(dataset)} записями.")
print("Доступные колонки:", dataset.column_names)
print("Разделение датасета на train (80%) и test (20%)...")
split_dataset = dataset.train_test_split(test_size=0.2, seed=42)
train_data = split_dataset['train']
test_data = split_dataset['test']
print("Датасет разделен.")
print(f"Размер обучающей выборки: {len(train_data)}")
print(f"Размер тестовой выборки: {len(test_data)}")

try:
    _ = train_data[0]['query']
    _ = train_data[0]['answer']

    test_documents = [item['answer'] for item in test_data]
    test_questions = [item['query'] for item in test_data]
    train_documents = [item['answer'] for item in train_data]
    train_questions = [item['query'] for item in train_data]

except KeyError as e:
    print(f"Ошибка: Ожидаемое поле {e} не найдено в датасете.")
    print("Пожалуйста, проверьте доступные колонки:")
    print(dataset.column_names)
    raise ValueError(f"Отсутствует необходимое поле в датасете: {e}")

if not test_questions or not test_documents or not train_questions or not train_documents:
     print("Ошибка: Одна из выборок (train/test вопросы/документы) пуста.")
     raise ValueError("Выборка пуста.")

train_size_limit = 5000
print(f"\nПодготовка данных для дообучения (используется {train_size_limit} примеров)...")

all_train_documents_list = train_documents

train_examples_contrastive = []
for i in range(min(train_size_limit, len(train_data))):
    item = train_data[i]
    query = item['query']
    positive_doc = item['answer']
    train_examples_contrastive.append(InputExample(texts=[query, positive_doc], label=1.0))
    negative_doc = random.choice(all_train_documents_list)
    while negative_doc == positive_doc and len(all_train_documents_list) > 1:
         negative_doc = random.choice(all_train_documents_list)

    if negative_doc != positive_doc or len(all_train_documents_list) == 1:
         train_examples_contrastive.append(InputExample(texts=[query, negative_doc], label=0.0))

print(f"Подготовлено {len(train_examples_contrastive)} примеров для Contrastive Loss.")

train_examples_triplet = []
for i in range(min(train_size_limit, len(train_data))):
    item = train_data[i]
    anchor = item['query']
    positive = item['answer']
    negative = random.choice(all_train_documents_list)
    while negative == positive and len(all_train_documents_list) > 1:
         negative = random.choice(all_train_documents_list)

    if negative != positive or len(all_train_documents_list) == 1:
        train_examples_triplet.append(InputExample(texts=[anchor, positive, negative]))

print(f"Подготовлено {len(train_examples_triplet)} примеров для Triplet Loss.")

model_name = "intfloat/multilingual-e5-base"
num_epochs = 1
train_batch_size = 16

print(f"\nДообучение модели с Contrastive Loss (эпох: {num_epochs}, батч: {train_batch_size}, примеров: {len(train_examples_contrastive)})...")
model_contrastive = SentenceTransformer(model_name, device=device)
train_dataloader_contrastive = DataLoader(train_examples_contrastive, shuffle=True, batch_size=train_batch_size)
train_loss_contrastive = losses.ContrastiveLoss(model=model_contrastive)

warmup_steps = int(len(train_dataloader_contrastive) * num_epochs * 0.1) if len(train_dataloader_contrastive) > 0 else 0
output_path_contrastive = "output/e5-contrastive-finetuned"
model_contrastive.fit(train_objectives=[(train_dataloader_contrastive, train_loss_contrastive)],
                      epochs=num_epochs,
                      warmup_steps=warmup_steps,
                      output_path=output_path_contrastive)
print("Дообучение с Contrastive Loss завершено.")

print(f"\nДообучение модели с Triplet Loss (эпох: {num_epochs}, батч: {train_batch_size}, примеров: {len(train_examples_triplet)})...")
model_triplet = SentenceTransformer(model_name, device=device)
train_dataloader_triplet = DataLoader(train_examples_triplet, shuffle=True, batch_size=train_batch_size)
train_loss_triplet = losses.TripletLoss(model=model_triplet, distance_metric=losses.TripletDistanceMetric.COSINE)

warmup_steps_triplet = int(len(train_dataloader_triplet) * num_epochs * 0.1) if len(train_dataloader_triplet) > 0 else 0
output_path_triplet = "output/e5-triplet-finetuned"
model_triplet.fit(train_objectives=[(train_dataloader_triplet, train_loss_triplet)],
                  epochs=num_epochs,
                  warmup_steps=warmup_steps_triplet,
                  output_path=output_path_triplet)
print("Дообучение с Triplet Loss завершено.")

def evaluate_model(model, test_questions, test_documents):
    print(f"\nОценка модели...")
    print("Векторизация тестовых вопросов и документов...")
    test_question_embeddings = model.encode(test_questions, convert_to_numpy=True, show_progress_bar=False)
    test_document_embeddings = model.encode(test_documents, convert_to_numpy=True, show_progress_bar=False)
    print("Векторизация завершена.")

    print(f"Форма эмбеддингов вопросов: {test_question_embeddings.shape}")
    print(f"Форма эмбеддингов документов: {test_document_embeddings.shape}")

    print("Расчет косинусной близости...")
    if test_question_embeddings.shape[0] == 0 or test_document_embeddings.shape[0] == 0:
        print("Ошибка: Пустые эмбеддинги тестовой выборки.")
        return {"MRR": 0, "Recall@1": 0, "Recall@3": 0, "Recall@10": 0}

    similarity_matrix = util.cos_sim(test_question_embeddings, test_document_embeddings).numpy()
    print("Расчет близости завершен.")

    print("Расчет метрик MRR и Recall@k...")
    mrr_scores = []
    recall_at_1 = 0
    recall_at_3 = 0
    recall_at_10 = 0
    num_test_questions = len(test_questions)

    if num_test_questions == 0:
        print("Ошибка: Количество тестовых вопросов равно нулю.")
        return {"MRR": 0, "Recall@1": 0, "Recall@3": 0, "Recall@10": 0}

    for i in range(num_test_questions):
        true_document_index = i
        question_similarity_scores = similarity_matrix[i]
        ranked_document_indices = np.argsort(question_similarity_scores)[::-1]
        rank = -1
        indices_of_true_doc = np.where(ranked_document_indices == true_document_index)[0]

        if indices_of_true_doc.size > 0:
            rank = indices_of_true_doc[0] + 1
            mrr_scores.append(1.0 / rank)
            if rank <= 1:
                recall_at_1 += 1
            if rank <= 3:
                recall_at_3 += 1
            if rank <= 10:
                recall_at_10 += 1

    mean_mrr = np.mean(mrr_scores) if mrr_scores else 0
    final_recall_at_1 = recall_at_1 / num_test_questions
    final_recall_at_3 = recall_at_3 / num_test_questions
    final_recall_at_10 = recall_at_10 / num_test_questions
    results = {
        "MRR": mean_mrr,
        "Recall@1": final_recall_at_1,
        "Recall@3": final_recall_at_3,
        "Recall@10": final_recall_at_10
    }
    return results

print("\n--- Результаты E5 Contrastive Fine-tuned ---")
try:
    model_contrastive_loaded = SentenceTransformer(output_path_contrastive, device=device)
    results_contrastive = evaluate_model(model_contrastive_loaded, test_questions, test_documents)
    print(results_contrastive)
except Exception as e:
    print(f"Ошибка при оценке модели с Contrastive Loss: {e}")
    results_contrastive = {"MRR": 0, "Recall@1": 0, "Recall@3": 0, "Recall@10": 0}

print("\n--- Результаты E5 Triplet Fine-tuned ---")
try:
    model_triplet_loaded = SentenceTransformer(output_path_triplet, device=device)
    results_triplet = evaluate_model(model_triplet_loaded, test_questions, test_documents)
    print(results_triplet)
except Exception as e:
    print(f"Ошибка при оценке модели с Triplet Loss: {e}")
    results_triplet = {"MRR": 0, "Recall@1": 0, "Recall@3": 0, "Recall@10": 0}

print(f"E5 Contrastive Fine-tuned: {results_contrastive}")
print(f"E5 Triplet Fine-tuned: {results_triplet}")

Weights & Biases отключен.
Используется устройство: cuda
Загрузка датасета sentence-transformers/natural-questions...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.28k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


train-00000-of-00001.parquet:   0%|          | 0.00/44.0M [00:00<?, ?B/s]

...

Дообучение с Contrastive Loss завершено.

Дообучение модели с Triplet Loss (эпох: 1, батч: 16, примеров: 5000)...


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss


Дообучение с Triplet Loss завершено.

--- Результаты E5 Contrastive Fine-tuned ---

Оценка модели...
Векторизация тестовых вопросов и документов...
Векторизация завершена.
Форма эмбеддингов вопросов: (20047, 768)
Форма эмбеддингов документов: (20047, 768)
Расчет косинусной близости...
Расчет близости завершен.
Расчет метрик MRR и Recall@k...
{'MRR': np.float64(0.7091966991209268), 'Recall@1': 0.5949019803461865, 'Recall@3': 0.7968773382551005, 'Recall@10': 0.9072180376116127}

--- Результаты E5 Triplet Fine-tuned ---

Оценка модели...
Векторизация тестовых вопросов и документов...
Векторизация завершена.
Форма эмбеддингов вопросов: (20047, 768)
Форма эмбеддингов документов: (20047, 768)
Расчет косинусной близости...
Расчет близости завершен.
Расчет метрик MRR и Recall@k...
{'MRR': np.float64(0.46524601601078924), 'Recall@1': 0.36259789494687483, 'Recall@3': 0.5241682047189106, 'Recall@10': 0.6566069736120118}
E5 Contrastive Fine-tuned: {'MRR': np.float64(0.7091966991209268), 'Recall@1'

1. Какие получились метрики? Что можно о них сказать?

E5 Contrastive Fine-tuned: {'MRR': np.float64(0.7091966991209268), 'Recall@1': 0.5949019803461865, 'Recall@3': 0.7968773382551005, 'Recall@10': 0.9072180376116127}

E5 Triplet Fine-tuned: {'MRR': np.float64(0.46524601601078924), 'Recall@1': 0.36259789494687483, 'Recall@3': 0.5241682047189106, 'Recall@10': 0.6566069736120118}

Метрики выше, чем у ванильного E5 и TF-IDF.

2. Что сработало лучше - Contrastive Loss или Triplet Loss? Почему?

Contrastive Loss стремится приблизить положительные пары (вопрос-ответ) и отдалить отрицательные пары (вопрос-негативный ответ).

Triplet Loss работает с тройками и стремится, чтобы эмбеддинг anchor был ближе к positive, чем к negative, с определенным отступом.

Лучше сработал первый вариант.

3. Стало ли лучше в сравнении с ванильным E5? Почему?

Да, стало лучше.

Почему стало лучше?

- Специфическое доменное обучение;

- Оптимизация под задачу поиска.

Таска 5

In [3]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer, InputExample, losses, util
from torch.utils.data import DataLoader
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import os

os.environ["WANDB_DISABLED"] = "true"
print("Weights & Biases отключен.")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Используется устройство: {device}")

print("Загрузка датасета sentence-transformers/natural-questions...")
try:
    dataset = load_dataset("sentence-transformers/natural-questions", split='train')
    print("Датасет загружен.")
except ValueError as e:
    print(f"Ошибка при загрузке датасета: {e}")
    print("Возможно, требуется обновить библиотеки datasets и fsspec: pip install --upgrade datasets fsspec")
    raise

print(f"Загружен сплит с {len(dataset)} записями.")
print("Доступные колонки:", dataset.column_names)

print("Разделение датасета на train (80%) и test (20%)...")
split_dataset = dataset.train_test_split(test_size=0.2, seed=42)
train_data = split_dataset['train']
test_data = split_dataset['test']
print("Датасет разделен.")
print(f"Размер обучающей выборки: {len(train_data)}")
print(f"Размер тестовой выборки: {len(test_data)}")

try:
    i = train_data[0]['query']
    j = train_data[0]['answer']

    test_documents = [item['answer'] for item in test_data]
    test_questions = [item['query'] for item in test_data]
    train_documents = [item['answer'] for item in train_data]
    train_questions = [item['query'] for item in train_data]


except KeyError as e:
    print(f"Ошибка: Ожидаемое поле {e} не найдено в датасете.")
    print("Пожалуйста, проверьте доступные колонки:")
    print(dataset.column_names)
    raise ValueError(f"Отсутствует необходимое поле в датасете: {e}")

if not test_questions or not test_documents or not train_questions or not train_documents:
     print("Ошибка: Одна из выборок (train/test вопросы/документы) пуста.")
     raise ValueError("Выборка пуста.")

train_size_limit = 5000
print(f"\nПодготовка данных для дообучения с Hard Negatives (используется {train_size_limit} примеров)...")

train_examples_hard_negatives = []
for i in range(min(train_size_limit, len(train_data))):
    item = train_data[i]
    anchor = item['query']
    positive = item['answer']
    train_examples_hard_negatives.append(InputExample(texts=[anchor, positive], label=i))

print(f"Подготовлено {len(train_examples_hard_negatives)} примеров для дообучения с Hard Negatives.")

model_name = "intfloat/multilingual-e5-base"
num_epochs = 1
train_batch_size = 64

def custom_pairwise_cos_sim(embeddings):
    return util.cos_sim(embeddings, embeddings)


print(f"\nДообучение модели с BatchHardTripletLoss (эпох: {num_epochs}, батч: {train_batch_size}, примеров: {len(train_examples_hard_negatives)})...")
model_hard_negatives = SentenceTransformer(model_name, device=device)
train_dataloader_hard_negatives = DataLoader(train_examples_hard_negatives, shuffle=True, batch_size=train_batch_size)
train_loss_hard_negatives = losses.BatchHardTripletLoss(model=model_hard_negatives, distance_metric=custom_pairwise_cos_sim, margin=0.5)

warmup_steps_hard_negatives = int(len(train_dataloader_hard_negatives) * num_epochs * 0.1) if len(train_dataloader_hard_negatives) > 0 else 0
output_path_hard_negatives = "output/e5-hard-negatives-finetuned"
model_hard_negatives.fit(train_objectives=[(train_dataloader_hard_negatives, train_loss_hard_negatives)],
                      epochs=num_epochs,
                      warmup_steps=warmup_steps_hard_negatives,
                      output_path=output_path_hard_negatives)
print("Дообучение с BatchHardTripletLoss завершено.")

def evaluate_model(model, test_questions, test_documents):
    print(f"\nОценка модели...")
    print("Векторизация тестовых вопросов и документов...")
    test_question_embeddings = model.encode(test_questions, convert_to_numpy=True, show_progress_bar=False)
    test_document_embeddings = model.encode(test_documents, convert_to_numpy=True, show_progress_bar=False)
    print("Векторизация завершена.")

    print(f"Форма эмбеддингов вопросов: {test_question_embeddings.shape}")
    print(f"Форма эмбеддингов документов: {test_document_embeddings.shape}")

    print("Расчет косинусной близости...")
    if test_question_embeddings.shape[0] == 0 or test_document_embeddings.shape[0] == 0:
        print("Ошибка: Пустые эмбеддинги тестовой выборки.")
        return {"MRR": 0, "Recall@1": 0, "Recall@3": 0, "Recall@10": 0}

    similarity_matrix = util.cos_sim(test_question_embeddings, test_document_embeddings).numpy()
    print("Расчет близости завершен.")

    print("Расчет метрик MRR и Recall@k...")
    mrr_scores = []
    recall_at_1 = 0
    recall_at_3 = 0
    recall_at_10 = 0
    num_test_questions = len(test_questions)

    if num_test_questions == 0:
        print("Ошибка: Количество тестовых вопросов равно нулю.")
        return {"MRR": 0, "Recall@1": 0, "Recall@3": 0, "Recall@10": 0}

    for i in range(num_test_questions):
        true_document_index = i
        question_similarity_scores = similarity_matrix[i]
        ranked_document_indices = np.argsort(question_similarity_scores)[::-1]
        rank = -1
        indices_of_true_doc = np.where(ranked_document_indices == true_document_index)[0]
        if indices_of_true_doc.size > 0:
            rank = indices_of_true_doc[0] + 1
            mrr_scores.append(1.0 / rank)
            if rank <= 1:
                recall_at_1 += 1
            if rank <= 3:
                recall_at_3 += 1
            if rank <= 10:
                recall_at_10 += 1

    mean_mrr = np.mean(mrr_scores) if mrr_scores else 0
    final_recall_at_1 = recall_at_1 / num_test_questions
    final_recall_at_3 = recall_at_3 / num_test_questions
    final_recall_at_10 = recall_at_10 / num_test_questions
    results = {
        "MRR": mean_mrr,
        "Recall@1": final_recall_at_1,
        "Recall@3": final_recall_at_3,
        "Recall@10": final_recall_at_10
    }
    return results

print("\n--- Результаты E5 BatchHardTripletLoss Fine-tuned ---")
try:
    model_hard_negatives_loaded = SentenceTransformer(output_path_hard_negatives, device=device)
    results_hard_negatives = evaluate_model(model_hard_negatives_loaded, test_questions, test_documents)
    print(results_hard_negatives)
except Exception as e:
    print(f"Ошибка при оценке модели с BatchHardTripletLoss: {e}")
    results_hard_negatives = {"MRR": 0, "Recall@1": 0, "Recall@3": 0, "Recall@10": 0}

print(f"E5 Triplet (Hard Negatives) Fine-tuned: {results_hard_negatives}")

Weights & Biases отключен.
Используется устройство: cuda
Загрузка датасета sentence-transformers/natural-questions...
Датасет загружен.
Загружен сплит с 100231 записями.
Доступные колонки: ['query', 'answer']
Разделение датасета на train (80%) и test (20%)...
Датасет разделен.
Размер обучающей выборки: 80184
Размер тестовой выборки: 20047

Подготовка данных для дообучения с Hard Negatives (используется 5000 примеров)...
Подготовлено 5000 примеров для дообучения с Hard Negatives.

Дообучение модели с BatchHardTripletLoss (эпох: 1, батч: 64, примеров: 5000)...


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss


Дообучение с BatchHardTripletLoss завершено.

--- Результаты E5 BatchHardTripletLoss Fine-tuned ---

Оценка модели...
Векторизация тестовых вопросов и документов...
Векторизация завершена.
Форма эмбеддингов вопросов: (20047, 768)
Форма эмбеддингов документов: (20047, 768)
Расчет косинусной близости...
Расчет близости завершен.
Расчет метрик MRR и Recall@k...
{'MRR': np.float64(0.7891481636514763), 'Recall@1': 0.6814485957998703, 'Recall@3': 0.8816780565670674, 'Recall@10': 0.9633361600239437}
E5 Triplet (Hard Negatives) Fine-tuned: {'MRR': np.float64(0.7891481636514763), 'Recall@1': 0.6814485957998703, 'Recall@3': 0.8816780565670674, 'Recall@10': 0.9633361600239437}


1. Какие получились метрики? Что можно о них сказать?

- Метрики выше по сравнению с обучением на случайных негативах. Более высокий MRR указывает на то, что правильный релевантный документ в среднем ранжируется выше.

- Более высокие значения Recall@k предполагают, что релевантный документ чаще находится среди топ k извлеченных результатов.

2. Стало ли лучше в сравнении с random negatives? Почему?

Да, производительность (измеренная с помощью MRR и Recall@k) улучшилась по сравнению с использованием случайных негативов. Это связано с тем, что "жесткие" негативы специально выбираются как сложные примеры – это документы, которые модели в данный момент трудно отличить от правильного ответа, будучи семантически похожими на запрос. Обучая модель отталкивать эти негативные примеры дальше от вектора запроса, одновременно подтягивая правильный положительный пример ближе, модель вынуждена учиться более тонким и устойчивым представлениям. Эта улучшенная способность различать релевантные и нерелевантные, но похожие документы напрямую приводит к лучшей производительности ранжирования в задачах поиска, что ведет к более высоким показателям MRR и Recall.